For my other works in this TPS;

https://www.kaggle.com/mustafacicek/tps-09-21-eda

https://www.kaggle.com/mustafacicek/tps-09-21-xgboost-0-81785

https://www.kaggle.com/mustafacicek/tps-09-21-lightgbm-0-81772

In [ ]:
import pandas  as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_auc_score

from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")

In [ ]:
train["missing"] = train.isnull().sum(axis = 1)
test["missing"] = test.isnull().sum(axis = 1)

In [ ]:
target = "claim"
predictors = [x for x in train.columns if x not in ["id", target]]

kf = KFold(n_splits = 5, shuffle = True, random_state = 666)
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 666)

In [ ]:
train[predictors] = train[predictors].fillna(train.groupby("missing")[predictors].transform("median"))
test[predictors] = test[predictors].fillna(train.groupby("missing")[predictors].transform("median"))

In [ ]:
scaler = StandardScaler()

train[predictors] = scaler.fit_transform(train[predictors])
test[predictors] = scaler.transform(test[predictors])

In [ ]:
X = train[predictors]
y = train[target]
test = test[predictors]

In [ ]:
params = {
    'max_depth': 12, 
    'max_leaf_nodes': 175, 
    'min_samples_leaf': 13646, 
    'l2_regularization': 0.4559366258442665
}

In [ ]:
oof_hist = np.zeros(len(X))
predictions_hist = np.zeros(len(test))
i = 1

for train_ix, test_ix in skf.split(X.values, y.values):
        
    print("\033[1m" + "Out of fold predictions generating for fold \033[94m {} \033[0m \n".format(i))
        
    train_X, train_y = X.values[train_ix], y.values[train_ix]
    test_X, test_y = X.values[test_ix], y.values[test_ix]
    
    model_hist = HistGradientBoostingClassifier(
        random_state = 666,
        max_iter = 40000,
        learning_rate = 0.025,
        validation_fraction = 0.1,
        early_stopping = True,
        n_iter_no_change = 200,
        scoring = "roc_auc",
        verbose = 0,
        **params
    )   
    
    model_hist.fit(
        train_X, 
        train_y,
    )
    
    oof_hist[test_ix] = oof_hist[test_ix] + model_hist.predict_proba(test_X)[:, 1]
    predictions_hist = predictions_hist + model_hist.predict_proba(test)[:, 1]
    
    print("\033[1mAUC for fold \033[91m{} \t\t\t \033[92m {} \033[0m \n".format(i, round(roc_auc_score(test_y, oof_hist[test_ix]), 5)))
    
    i = i + 1
    
print("\033[1mAUC for Training Set: \t\t \033[92m {} \033[0m \n".format(round(roc_auc_score(y, oof_hist), 5)))

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")
submission[target] = predictions_hist / 5
submission.to_csv("submission.csv", index = False)
submission

In [ ]:
np.save("oof_hist.npy", oof_hist)
np.save("preds_hist.npy", predictions_hist)